
# Classification d'Images par Deep Learning

Ce notebook implémente un modèle de classification d'images multi-classes utilisant TensorFlow/Keras.
Nous allons suivre les étapes suivantes :
1. Préparation et chargement des données
2. Création du modèle
3. Entraînement
4. Évaluation



## 1. Importation des Bibliothèques Requises


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
import os

## 2. Création du Générateur de Données Personnalisé

Cette classe permet de charger et prétraiter les images par lots, ce qui est plus efficace en termes de mémoire que de charger toutes les images en une fois.

In [4]:
class DataGenerator(Sequence):
    def __init__(self, csv_path, img_dir, batch_size=32, img_size=(224, 224), shuffle=True):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.class_columns = self.df.columns[1:].tolist()
        self.n = len(self.df)
        self.indexes = np.arange(self.n)
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(self.n / self.batch_size))

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, self.n)
        batch_indexes = self.indexes[start_idx:end_idx]
          
        batch_x = []
        batch_y = []
        
        for i in batch_indexes:
            row = self.df.iloc[i]
            try:
                img_path = os.path.join(self.img_dir, row['filename'])
                if os.path.exists(img_path):
                    img = load_img(img_path, target_size=self.img_size)
                    img_array = img_to_array(img)
                    img_array = img_array / 255.0
                    batch_x.append(img_array)
                    label = row[self.class_columns].values.astype('float32')
                    batch_y.append(label)
            except Exception as e:
                print(f"Error loading image {row['filename']}: {str(e)}")
                continue

        if len(batch_x) == 0:
            return self.__getitem__((idx + 1) % len(self))

        return np.array(batch_x), np.array(batch_y)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

## 3. Définition de l'Architecture du Modèle

Création d'un modèle CNN utilisant l'API Sequential de Keras

In [5]:
def create_model(num_classes, input_shape=(224, 224, 3)):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid')
    ])
    return model

## 4. Configuration des Répertoires et Paramètres


In [6]:
# Define data directories and parameters
train_dir = 'data/train'
valid_dir = 'data/valid'
test_dir = 'data/test'
batch_size = 16  # Reduced batch size for better memory management

print("Creating data generators...")

Creating data generators...


## 5. Création des Générateurs de Données


In [7]:
# Create data generators
train_generator = DataGenerator(
    os.path.join(train_dir, '_classes.csv'),
    train_dir,
    batch_size=batch_size
)

valid_generator = DataGenerator(
    os.path.join(valid_dir, '_classes.csv'),
    valid_dir,
    batch_size=batch_size,
    shuffle=False
)

test_generator = DataGenerator(
    os.path.join(test_dir, '_classes.csv'),
    test_dir,
    batch_size=batch_size,
    shuffle=False
)

## 6. Détermination du Nombre de Classes


In [8]:
# Get number of classes from the first batch
print("Chargement du premier lot pour déterminer le nombre de classes...")
_, first_batch_y = train_generator[0]
num_classes = first_batch_y.shape[1]
print(f"Nombre de classes : {num_classes}")

Chargement du premier lot pour déterminer le nombre de classes...
Nombre de classes : 14


## 7. Création et Compilation du Modèle


In [9]:
# Create and compile model
print("Création et compilation du modèle...")
model = create_model(num_classes)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

Création et compilation du modèle...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     5,537,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 14)             │         1,806 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,596,046 (21.35 MB)

 Trainable params: 5,596,046 (21.35 MB)

 Non-trainable params: 0 (0.00 B)

## 8. Entraînement du Modèle


In [11]:
# Train model
print("Début de l'entraînement...")
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    verbose=1
)

Début de l'entraînement...


c:\Users\chaim\Desktop\anomaly_detection\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20



KeyboardInterrupt



## 9. Évaluation et Sauvegarde du Modèle


In [12]:
# Evaluate model
print("Évaluation du modèle...")
test_results = model.evaluate(test_generator)
print(f"\nPerte de test, Précision de test:", test_results)
# Save model
print("Sauvegarde du modèle...")
model.save('image_classifier_model.h5')

print("Entraînement terminé avec succès!")

Évaluation du modèle...
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 300ms/step - accuracy: 0.0176 - loss: 0.6905



Perte de test, Précision de test: [0.690777599811554, 0.025157231837511063]
Sauvegarde du modèle...
Entraînement terminé avec succès!
